In [1]:
%%time
import malaya

CPU times: user 11.4 s, sys: 798 ms, total: 12.2 s
Wall time: 12.4 s


In [2]:
string = 'y u xsuka makan HUSEIN kt situ tmpt'
another = 'i mmg xska mknn HUSEIN kampng tempt'

## Load basic normalizer

In [3]:
malaya.normalize.basic(string)

'kenapa awak xsuka makan Husein kt situ tmpt'

## Load fuzzy normalizer

In [4]:
malays = malaya.load_malay_dictionary()
normalizer = malaya.normalize.fuzzy(malays)

In [5]:
normalizer.normalize(string)

'kenapa awak tak suka makan Husein kat situ tempat'

In [6]:
normalizer.normalize(another)

'saya memang tak saka makanan Husein kampung tempat'

## Load spell normalizer

In [7]:
normalizer = malaya.normalize.spell(malays)

#### To list all selected words during normalize

In [8]:
normalizer.normalize(string,debug=True)

[(('ampit', False), 20), (('tempat', False), 20), (('tempo', False), 15), (('tat', False), 11), (('tapa', False), 10), (('ampu', False), 15), (('impi', False), 15), (('amput', False), 20), (('taut', False), 15), (('tuat', False), 15), (('ampe', False), 15), (('top', False), 11), (('umut', False), 21), (('tampu', False), 15), (('timpa', False), 20), (('temut', False), 15), (('tut', False), 16), (('empat', True), 15), (('tumit', False), 20), (('amit', False), 21), (('topi', False), 10), (('umpat', True), 20), (('tepi', False), 10), (('tumpat', True), 24), (('umat', False), 21), (('ampo', False), 15), (('tepu', False), 10), (('tipu', False), 15), (('empu', False), 10), (('tip', False), 16), (('tempa', False), 15), (('tuit', False), 15), (('tampa', False), 15), (('tepet', False), 15), (('emat', False), 15), (('tamat', False), 20), (('taat', False), 15), (('amat', False), 21), (('tampi', True), 15), (('tapi', False), 10), (('tempe', False), 15), (('mat', False), 16), (('tumpu', False), 20),

'kenapa awak tak suka makan Husein kat situ umat'

## Load deep learning

This model is not perfect, really suggest you to use other models

In [9]:
normalizer = malaya.normalize.deep_model()
normalizer.normalize(string)

'eye uau tak suka makan unsein kati situ tumpat'